# Improving measurements by measuring Pauli products in multiple groups

One of the most common strategies for measuring the expectation value of a complicated observable is to partition the observable into groups of mutually comutative Pauil products \[[J. Chem. Theory Comput. **16**(4), 2400–2409 (2020)](https://doi.org/10.1021/acs.jctc.0c00008)\]. The observable expectation value is then obtained by In this scheme, The total number of measurements required to obtain the expectation value is proportional to the sum of variances of the Pauli product groups \[[Quantum **5**, 385 (2021)](https://doi.org/10.22331/q-2021-01-20-385)\]. The method introduced here achieves a severalfold reduction in the number of required measurements by measuring Pauli products in all possible groups \[[arXiv:2201.01471](
https://doi.org/10.48550/arXiv.2201.01471
)\]. This novel measurement technique is called the *iterative coefficient splitting* (ICS) method.

## How to Use:
The following demonstrates how the ICS method can be used in tequila. See [MeasurementGroups](./MeasurementGroups.ipynb) for further details on transforming the group of commutative Pauli product terms into Pauli-z operators, measurable on a quantum computer.

### Setting up the test Hamiltonian and covariance dictionary:
We demonstrate ICS using a simple Hamiltonian for H$_2$ in STO-3G basis set. For ICS, we allocate coefficients and number of samples per group according to the approximate covariances. In this simple example, we use the FCI wavefunction to evaluate these. However, in a more realistic computations, one would use a classically efficient proxy, e.g., Hartree-Fock or CISD wavefunction to approximate the covariances.

For this test example, we use a simple Pauli rotation with a fixed angle as the VQE unitary. 

In [41]:
# Needs pyscf or psi4 installed:
# pip install pyscf
# tequila version needs to be > 1.8.3 or from devel branchimport tequila as tq
from tequila.hamiltonian import QubitHamiltonian, PauliString
from tequila.grouping.binary_rep import BinaryPauliString, BinaryHamiltonian
import numpy as np
import pickle

def prepare_test_hamiltonian():
    '''
    Return a test hamiltonian.
    '''
    trafo = "BravyiKitaev"
    mol = tq.chemistry.Molecule(
                            geometry="H 0.0 0.0 0.0 \n H 0.0 0.0 1.",
                            basis_set="sto3g",
                            transformation=trafo,
                            backend='pyscf'
                                 )
    H = mol.make_hamiltonian()

    U = mol.make_ansatz(name="SPA", edges=[(0,1)])
    E = tq.ExpectationValue(H=H, U=U)
    result = tq.minimize(E, silent=True)
    wfn = tq.simulate(U, variables=result.variables)\
    
    Hbin = BinaryHamiltonian.init_from_qubit_hamiltonian(H)
    return Hbin, wfn, len(Hbin.binary_terms) - 1

def prepare_cov_dict(H):
    '''
    Return the covariance dictionary containing Cov(P1, P2). 
    In a practical calculation, this covariance dictionary would be built from
    a Hartree-Fock or configuration interaction singles and doulbes (CISD) 
    wavefunction. Here, we use the CISD wavefunction.
    '''
    terms = H.binary_terms
    cov_dict = {}
    with open("h2_cisd_wfn.pkl", "rb") as f:
        wfn0 = tq.QubitWaveFunction(pickle.load(f))
    for idx, term1 in enumerate(terms):
        for term2 in terms[idx:]:
            pw1 = BinaryPauliString(term1.get_binary(), 1.0)
            pw2 = BinaryPauliString(term2.get_binary(), 1.0)
            op1 = QubitHamiltonian.from_paulistrings(pw1.to_pauli_strings())
            op2 = QubitHamiltonian.from_paulistrings(pw2.to_pauli_strings())
            if pw1.commute(pw2):
                prod_op = op1 * op2
                cov_dict[(term1.binary_tuple(), term2.binary_tuple())] = wfn0.inner(prod_op(wfn0)) - wfn0.inner(op1(wfn0)) * wfn0.inner(op2(wfn0))
    return cov_dict

Hbin, wfn0, n_paulis = prepare_test_hamiltonian()
cov_dict = prepare_cov_dict(Hbin)

U = tq.gates.ExpPauli(angle="a", paulistring=tq.PauliString.from_string('X(0)Y(1)'))
variables = {"a": 0.35 * 2 * np.pi}

converged SCF energy = -1.06610864931794


### Defining the options for ICS:

To use ICS, define the options as following. Note that cov_dict can be precomputed using a separate subscript and should contain the approximate covariances, Cov(P1, P2). The measurement groups are optimized according to cov_dict.

In [17]:
options = {"method":"osi", "condition": "fc", "cov_dict":cov_dict}

### Comparing the performance

In the following, we demonstrate the performance of ICS by comparing it with simple measurement schemes. Note that the total number of samples in the conventional method is samples * n_paulis, whereas the total number of samples is # in "auto-#" in ICS (see below).
We will compute the **average error** from **1000 measurements** to demonstrate ICS's performance.

In [43]:
print("=========================================")
print("Unoptimized measurement scheme")
print("=========================================")
e_ori = tq.ExpectationValue(H=Hbin.to_qubit_hamiltonian(), U=U)
print(e_ori)
print("=========================================")
print("Optimized measurement scheme")
print("=========================================")
e_ics = tq.ExpectationValue(H=Hbin.to_qubit_hamiltonian(), U=U, optimize_measurements=options)
print(e_ics)
print("=========================================")

result_ori = tq.simulate(e_ori, variables)
print("Benchmark Energy:", result_ori)

n_meas = 1000

exp_ics = np.zeros(n_meas) 
exp_no_opt = np.zeros(n_meas)

compiled_ics = tq.compile(e_ics)
compiled_no_opt = tq.compile(e_ori)

for repeat in range(n_meas):
    exp_ics[repeat] = compiled_ics(variables, samples="auto-1000")
    exp_no_opt[repeat] = compiled_no_opt(variables, samples=int(1000/n_paulis))

print("Energy measured with ICS:", np.average(exp_ics))
print("Energy measured without optimization:", np.average(exp_no_opt))
print("Factor of reduction in average error by using ICS:", 
      np.average( np.abs(exp_no_opt - result_ori) ) 
      / np.average( np.abs(exp_ics - result_ori) ) )

Unoptimized measurement scheme
Objective with 1 unique expectation values
total measurements = 15
variables          = [a]
types              = not compiled
Optimized measurement scheme
Objective with 2 unique expectation values
total measurements = 2
variables          = [a]
types              = not compiled
Benchmark Energy: -0.32684417286792544
Energy measured with ICS: -0.32726723650097844
Energy measured without optimization: -0.3282812211667907
Factor of reduction in average error by using ICS: 2.581172435465373
